In [ ]:
#======================================================================
#
#     This routine solves an infinite horizon growth model 
#     with dynamic programming and sparse grids
#
#     The model is described in Scheidegger & Bilionis (2017)
#     https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2927400
#
#     external libraries needed:
#     - IPOPT (https://projects.coin-or.org/Ipopt)
#     - PYIPOPT (https://github.com/xuy/pyipopt)
#     - TASMANIAN (http://tasmanian.ornl.gov/)
#
#     Simon Scheidegger, 11/16 ; 07/17
#======================================================================

import nonlinear_solver_initial as solver     #solves opt. problems for terminal VF
import nonlinear_solver_iterate_shock as solviter   #solves opt. problems during VFI
from parameters import *                      #parameters of model
import interpolation as interpol              #interface to sparse grid library/terminal VF
import interpolation_iter_shock as interpol_iter    #interface to sparse grid library/iteration
import postprocessing as post                 #computes the L2 and Linfinity error of the model

import TasmanianSG                            #sparse grid library
import numpy as np


#======================================================================

# Shocks
theta_list = [0.9, 0.95, 1., 1.05, 1.1]

# Initial value function
valnew_list = []

if (numstart==0):
    for i in range(len(theta_list)):
        valnew_list.append(TasmanianSG.TasmanianSparseGrid())
        valnew_list[i] = interpol.sparse_grid(n_agents, iDepth)
        
valold_list=valnew_list
        
# Value function iteration
for i in range(numstart, numits):
    valnew_list = []
    for ii in range(len(theta_list)):
        print('i = {}'.format(i))
        
        valnew_list.append(TasmanianSG.TasmanianSparseGrid())
        if (i % 5 != 0) or (i == 0):
            print('no adaption loop')
            print('ii = {}'.format(ii))
            valnew_list[ii] = interpol_iter.sparse_grid_iter(valold_list[ii], theta_list[ii], valold_list)
        else:
            print('adaption loop')
            print('ii = {}'.format(ii))
            valnew_list[ii] = interpol_iter.sparse_grid_iter(valold_list[ii], theta_list[ii], valold_list, adaptive = True)
        
    valold_list = valnew_list
    
#======================================================================
print( "===============================================================")
print( " " )
print( " Computation of a growth model of dimension ", n_agents ," finished after ", numits, " steps")
print( " " )
print( "===============================================================")
#======================================================================

# compute errors   
avg_err=post.ls_error(n_agents, numstart, numits, No_samples)

#======================================================================
print( "===============================================================")
print( " ")
print( " Errors are computed -- see errors.txt")
print( " ")
print( "===============================================================")
#======================================================================